In [ ]:
!pip install pulp

!sudo apt-get install coinor-cbc glpk-utils coinor-clp


Reading package lists... Done
Building dependency tree       
Reading state information... Done
coinor-cbc is already the newest version (2.9.9+repack1-1).
coinor-clp is already the newest version (1.16.11+repack1-1).
glpk-utils is already the newest version (4.65-1).
0 upgraded, 0 newly installed, 0 to remove and 21 not upgraded.


In [1]:
### kod sprawdza czy instalacja się udała
import pulp

print(pulp.__version__)

2.3.1


In [2]:
pulp.pulpTestAll()

..

Solver <class 'pulp.apis.coin_api.PULP_CBC_CMD'> unavailable
Solver <class 'pulp.apis.cplex_api.CPLEX_DLL'> unavailable
Solver <class 'pulp.apis.cplex_api.CPLEX_CMD'> unavailable
Solver <class 'pulp.apis.cplex_api.CPLEX_PY'> unavailable
Solver <class 'pulp.apis.coin_api.COIN_CMD'> unavailable
Solver <class 'pulp.apis.coin_api.COINMP_DLL'> unavailable
Solver <class 'pulp.apis.glpk_api.GLPK_CMD'> available
Solver <class 'pulp.apis.xpress_api.XPRESS'> unavailable
Solver <class 'pulp.apis.gurobi_api.GUROBI'> unavailable
Solver <class 'pulp.apis.gurobi_api.GUROBI_CMD'> unavailable
Solver <class 'pulp.apis.glpk_api.PYGLPK'> unavailable
Solver <class 'pulp.apis.coin_api.YAPOSIB'> unavailable
Solver <class 'pulp.apis.choco_api.PULP_CHOCO_CMD'> unavailable
Solver <class 'pulp.apis.choco_api.CHOCO_CMD'> unavailable
Solver <class 'pulp.apis.mipcl_api.MIPCL_CMD'> unavailable
Solver <class 'pulp.apis.mosek_api.MOSEK'> unavailable
Solver <class 'pulp.apis.scip_api.SCIP_CMD'> unavailable
	 Testing in

..........

	 Testing maximize continuous LP solution
	 Testing continuous LP solution - export JSON
	 Testing continuous LP solution - export solver dict
	 Testing continuous LP solution - export solver JSON
	 Testing makeDict general behavior
	 Testing makeDict default value behavior
	 Testing the availability of the function pulpTestAll
	 Testing zero subtraction
	 Testing inconsistent lp solution


.......

...

........

	 Testing elastic constraints (penalty unchanged)
	 Testing elastic constraints (penalty unbounded)


...........................

{'a': 53.0, 'b': 45.3, 'c': 459.2}


............
----------------------------------------------------------------------
Ran 86 tests in 1.955s

OK


# Zadania:

1. przeczytaj materiały dotyczące pulp ze strony internetowej https://coin-or.github.io/pulp/main/index.html oraz przejdź przez blending problem (przepisz do swojego notebooka)

2. w blending problem dodaj zmienną decyzyjną. Stwórz zmienną która pojawi się w końcowym składzie produktu (tj. wartość zmiennej będzie miała niezerową postać)

3. Dodaj ograniczenie co do zawartości witaminy B 

Zawartość:

| Nazwa      | zawartość ug/100g |
|------------|-------------------|
| chicken    | 0.3               |
| beef       | 2.6               |
| mutton     | 2.6               |
| rice       | 0                 |
| wheat bran | 0                 |
| gel        | 0                 |
| sardines   | 8.9               |

Kot ma zjeść minimum 0.24 ug na kilogram masy więc około
0.8 ug dziennie

4. Na końcu ma pojawiać się zdanie podsumowujące wynik, w stylu:
Końcowy produkt będzie kosztować {}, będzie miał w składzie {nazwa}: {} %, {nastepna nazwa}: {} %

Takie zdanie można skleić z części generowanych automatycznie:



In [3]:
sklejone_produkt_cena = [":".join((produkt, str(cena))) for produkt,cena in zip(["jabłko","banan"],[5,10])]
print(sklejone_produkt_cena)
sklejona_calosc = ", ".join(sklejone_produkt_cena)

print(sklejona_calosc)





['jabłko:5', 'banan:10']
jabłko:5, banan:10


In [3]:
"""
The Simplified Whiskas Model Python Formulation for the PuLP Modeller

Authors: Antony Phillips, Dr Stuart Mitchell  2007
"""
# Import PuLP modeler functions
from pulp import *

# Create the 'prob' variable to contain the problem data
prob = LpProblem("The Whiskas Problem",LpMinimize)

# The 2 variables Beef and Chicken are created with a lower limit of zero
x1=LpVariable("ChickenPercent",0,None)
x2=LpVariable("BeefPercent",0)
x3=LpVariable("SardinesPercent",0)

# The objective function is added to 'prob' first
prob += 0.013*x1 + 0.008*x2 + 0.010*x3, "Total Cost of Ingredients per can"

# The five constraints are entered
prob += x1 + x2 + x3 == 100, "PercentagesSum"
prob += 0.100*x1 + 0.200*x2 + 0.150*x3 >= 8.0, "ProteinRequirement"
prob += 0.080*x1 + 0.010*x2 + 0.800*x3 >= 6.0, "FatRequirement"
prob += 0.001*x1 + 0.005*x2 + 0.002*x3 <= 2.0, "FibreRequirement"
prob += 0.002*x1 + 0.005*x2 + 0.004*x3 <= 0.4, "SaltRequirement"
prob += 0.030*x1 + 0.260*x2 + 0.500*x3 >= 0.8, "Vitamin B Requirent"

# The problem data is written to an .lp file
prob.writeLP("WhiskasModel.lp")

# The problem is solved using PuLP's choice of Solver
prob.solve()

# The status of the solution is printed to the screen
print("Status:", LpStatus[prob.status])

# Each of the variables is printed with it's resolved optimum value

for v in prob.variables():
    print(v.name, "=", v.varValue)
  

#The optimised objective function value is printed to the screen
print("Total Cost of Ingredients per can = ", value(prob.objective))    


sklejoneSkladnikZawarosc = ["=".join((v.name, str(v.varValue))) for v in prob.variables()]
sklejona_calosc = ", ".join(sklejoneSkladnikZawarosc)

    
print("Końcowy produkt będzie kosztować:", value(prob.objective), " będzie miał w składzie:",sklejona_calosc)


Status: Optimal
BeefPercent = 64.347826
ChickenPercent = 32.173913
SardinesPercent = 3.4782609
Total Cost of Ingredients per can =  0.967826086
Końcowy produkt będzie kosztować: 0.967826086  będzie miał w składzie: BeefPercent=64.347826, ChickenPercent=32.173913, SardinesPercent=3.4782609


In [5]:
"""
The Full Whiskas Model Python Formulation for the PuLP Modeller

Authors: Antony Phillips, Dr Stuart Mitchell  2007
"""

# Import PuLP modeler functions
from pulp import *

# Creates a list of the Ingredients
Ingredients = ['CHICKEN', 'BEEF', 'MUTTON', 'RICE', 'WHEAT', 'GEL','SARDINES']

# A dictionary of the costs of each of the Ingredients is created
costs = {'CHICKEN': 0.013, 
         'BEEF': 0.008, 
         'MUTTON': 0.010, 
         'RICE': 0.002, 
         'WHEAT': 0.005, 
         'GEL': 0.001,
         'SARDINES': 0.003}

# A dictionary of the protein percent in each of the Ingredients is created
proteinPercent = {'CHICKEN': 0.100, 
                  'BEEF': 0.200, 
                  'MUTTON': 0.150, 
                  'RICE': 0.000, 
                  'WHEAT': 0.040, 
                  'GEL': 0.000,
                  'SARDINES': 0.150}

# A dictionary of the fat percent in each of the Ingredients is created
fatPercent = {'CHICKEN': 0.080, 
              'BEEF': 0.100, 
              'MUTTON': 0.110, 
              'RICE': 0.010, 
              'WHEAT': 0.010, 
              'GEL': 0.000,
              'SARDINES': 0.80}
# A dictionary of the fibre percent in each of the Ingredients is created
fibrePercent = {'CHICKEN': 0.001, 
                'BEEF': 0.005, 
                'MUTTON': 0.003, 
                'RICE': 0.100, 
                'WHEAT': 0.150, 
                'GEL': 0.000,
                'SARDINES': 0.020}

# A dictionary of the salt percent in each of the Ingredients is created
saltPercent = {'CHICKEN': 0.002, 
               'BEEF': 0.005, 
               'MUTTON': 0.007, 
               'RICE': 0.002, 
               'WHEAT': 0.008, 
               'GEL': 0.000,
            'SARDINES': 0.008}


ViteaminBPercent = {'CHICKEN': 0.003, 
               'BEEF': 0.026, 
               'MUTTON': 0.026, 
               'RICE': 0.000, 
               'WHEAT': 0.000, 
               'GEL': 0.000,
               'SARDINES': 0.089}




# Create the 'prob' variable to contain the problem data
prob = LpProblem("The Whiskas Problem", LpMinimize)

# A dictionary called 'ingredient_vars' is created to contain the referenced Variables
ingredient_vars = LpVariable.dicts("Ingr",Ingredients,0)

# The objective function is added to 'prob' first
prob += lpSum([costs[i]*ingredient_vars[i] for i in Ingredients]), "Total Cost of Ingredients per can"

# The five constraints are added to 'prob'
prob += lpSum([ingredient_vars[i] for i in Ingredients]) == 100, "PercentagesSum"
prob += lpSum([proteinPercent[i] * ingredient_vars[i] for i in Ingredients]) >= 8.0, "ProteinRequirement"
prob += lpSum([fatPercent[i] * ingredient_vars[i] for i in Ingredients]) >= 6.0, "FatRequirement"
prob += lpSum([fibrePercent[i] * ingredient_vars[i] for i in Ingredients]) <= 2.0, "FibreRequirement"
prob += lpSum([saltPercent[i] * ingredient_vars[i] for i in Ingredients]) <= 0.4, "SaltRequirement"
prob += lpSum([ViteaminBPercent[i] * ingredient_vars[i] for i in Ingredients]) <= 0.4, "VitaminBRequirement"

# The problem data is written to an .lp file
prob.writeLP("WhiskasModel.lp")

# The problem is solved using PuLP's choice of Solver
prob.solve()

# The status of the solution is printed to the screen
print("Status:", LpStatus[prob.status])

# Each of the variables is printed with it's resolved optimum value
for v in prob.variables():
    print(v.name, "=", v.varValue)

# The optimised objective function value is printed to the screen
print("Total Cost of Ingredients per can = ", value(prob.objective))   

sklejoneSkladnikZawarosc = ["=".join((v.name, str(v.varValue))) for v in prob.variables()]
sklejona_calosc = ", ".join(sklejoneSkladnikZawarosc)

    
print("Końcowy produkt będzie kosztować:", value(prob.objective), " będzie miał w składzie:",sklejona_calosc)

Status: Optimal
Ingr_BEEF = 6.9316901
Ingr_CHICKEN = 60.431346
Ingr_GEL = 19.562831
Ingr_MUTTON = 0.0
Ingr_RICE = 0.0
Ingr_SARDINES = 0.43238223
Ingr_WHEAT = 12.64175
Total Cost of Ingredients per can =  0.92512974649
Końcowy produkt będzie kosztować: 0.92512974649  będzie miał w składzie: Ingr_BEEF=6.9316901, Ingr_CHICKEN=60.431346, Ingr_GEL=19.562831, Ingr_MUTTON=0.0, Ingr_RICE=0.0, Ingr_SARDINES=0.43238223, Ingr_WHEAT=12.64175
